In [3]:
from pyspark.sql import Row

sc = spark.sparkContext

# Load a text file and convert each line to a Row.
lines = sc.textFile("bilireply.txt")
parts = lines.map(lambda l: l.split(","))
# reply = parts.map(lambda p: Row(url,video_code,page,rpid,oid,mid,parent,root,co_unt,rcount,floor,ctime,love,uname,sex,sign,rank,current_level,mess_age,plat,device,crawl_time)))
reply = parts.map(lambda p: Row(url=p[0],video_code_r=int(p[1]),page=p[2],rpid=p[3],oid=p[4],mid=p[5],parent=p[6],root=p[7],co_unt=p[8],rcount=p[9],floor=p[10],ctime=p[11],love=int(p[12]),uname=p[13],sex=p[14],sign=p[15],rank=p[16],current_level=p[17],mess_age=p[18],plat=p[19],device=p[20],crawl_time=p[21]))

# ((url,int(video_code),page,rpid,oid,mid,parent,root,co_unt,rcount,floor,ctime,love,uname,sex,sign,rank,current_level,mess_age,plat,device,crawl_time))

In [4]:
# Infer the schema, and register the DataFrame as a table.
# schemaReply = spark.createDataFrame(reply)
df = spark.createDataFrame(reply)

df.createOrReplaceTempView("reply")


In [ ]:
# df
# SQL can be run over DataFrames that have been registered as a table.
# love2sql = "SELECT video_code,mess_age,love FROM reply where love>=2 limit 100"
# goodReply = spark.sql(love2sql)
# goodReply.collect()
df.printSchema()
# df.select(df['video_code_r'],df['love'],df['mess_age']).show()
# df.filter(df['love']>1).show()

In [7]:
Try_TokDF = spark.sql("select video_code_r,mess_age from reply order by video_code_r limit 100")
# Try_TokDF.show()

In [9]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
import jieba

tokenizer = jieba.Tokenizer(dictionary=DEFAULT_DICT, inputCol="sentence", outputCol="words")




NameError: name 'DEFAULT_DICT' is not defined

In [ ]:
countTokens = udf(lambda words: len(words), IntegerType())

# tokenized = tokenizer.transform(Try_TokDF)
# tokenized.select("mess_age", "words").withColumn("tokens", countTokens(col("words"))).show(truncate=False)

regexTokenized = regexTokenizer.transform(Try_TokDF)
regexTokenized.select("mess_age", "words").withColumn("tokens", countTokens(col("words"))).show(truncate=False)

## 基于RDD

In [9]:
reply

PythonRDD[7] at RDD at PythonRDD.scala:48

In [10]:
reply.count()

3030

In [11]:
# oneLoveDocument=df.rdd.filter(lambda l: l.love == 1)
# oneLoveDocument.count()

In [12]:
# oneLoveDocument.collect()

In [13]:
# twoLoveDocument=df.rdd.filter(lambda l: l.love == 2)
# twoLoveDocument.count()

In [14]:
# twoLoveDocument.collect()

In [15]:
# threeLoveDocument=df.rdd.filter(lambda l: l.love >= 3)
# threeLoveDocument.count()

## Combine the reply message

In [16]:
T1 = spark.sql("select video_code_r,mess_age FROM reply order by video_code_r")
# T1.show()

In [17]:
# spark.sql("SELECT * FROM global_temp.T1").show()
# T1.createGlobalTempView("T1")
import pyspark.sql.functions as F
combinedDF = T1.groupBy('video_code_r').agg(F.collect_list('mess_age'))
# combinedDF.show(10,False)
# combinedDF

# join the video information

In [18]:
from pyspark.sql import Row

sc = spark.sparkContext
# Load a text file and convert each line to a Row.
lines = sc.textFile("bilivideo.log")
parts = lines.map(lambda l: l.split(","))
video = parts.map(lambda p: Row(video_url=p[0], video_code=int(p[1]), publish_time=p[2], category1=p[3], category2=p[4], video_uploader=p[5],
                              uploader_id=p[6], uploader_upload_videos=p[7], up_sex=p[8], up_birth=p[9], up_fans=p[10],
                              up_friend=p[11], up_attention=p[12], up_level=p[13], up_plate=p[14], up_pla_level=p[15],up_pla_condition=[16],
                              view=int(p[17]),danmaku=int(p[18]),reply=p[19],favorite=p[20],coin=p[21],share=p[22],now_rank=p[23],his_rank=p[24],like_num=p[25],crawl_time=p[26]))

# Infer the schema, and register the DataFrame as a table.
# schemaVideo = spark.createDataFrame(video)
# schemaVideo.createOrReplaceTempView("video")
# vdf = spark.sql("SELECT video_code,view,danmaku,coin,favorite,like_num,reply,up_fans FROM video")


In [19]:
# Infer the schema, and register the DataFrame as a table.
schemaVideo = spark.createDataFrame(video)
schemaVideo.createOrReplaceTempView("video")
vdf = spark.sql("SELECT video_code,view,danmaku FROM video")

In [20]:
joinedDF = vdf.join(combinedDF, combinedDF.video_code_r == vdf.video_code)
# joinedDF.show()

In [21]:
joinedDF.count()

636

In [22]:
from functools import reduce
from pyspark.sql import DataFrame

# reduce(DataFrame.drop, ['GpuName','GPU1_TwoPartHwID'], joinedDF)
joinedDF = reduce(DataFrame.drop, ['video_code_r'] , joinedDF)

In [23]:
joinedDF.show()

+----------+-----+-------+----------------------+
|video_code| view|danmaku|collect_list(mess_age)|
+----------+-----+-------+----------------------+
|  14245770|  274|      2|  [ '粉嫩嫩的好可爱呀ヾ≧O≦〃嗷...|
|  14246196|   51|      1|  [ '今天终于补考完了，一会下单，...|
|  14247593| 1018|      0|  [ '原来是十诫石板是这么用的啊……']|
|  14245696|   22|      0|         [ '扎zn了，老fe']|
|  14246555|  126|      3|          [ '第一嘻嘻･∀･']|
|  14246820| 1439|     41|  [ '哇⊙o⊙哇',  '听up主...|
|  14246972|  459|     12|  [ '学生党没时间看直播啊。', ...|
|  14247195|  310|      0|              [ '第一？']|
|  14247454| 1792|     31|  [ '封面是啥',  '厉害厉害'...|
|  14245959|14472|    432|  [ '觉得大胖像黄渤，我老公说像孙...|
|  14246902|   62|      0|              [ '第一！']|
|  14247336|  799|     19|  [ '挨炮主，你以后会一直录制走a...|
|  14246764|  915|      2|  [ '料太多卷不起来的我心情复杂又...|
|  14246814|  275|      3|  [ 'up我们做朋友吧_´□`」 ...|
|  14246007|  884|      4|  [ '现版本三大弱c猴子小鱼人小黑...|
|  14246223| 1106|     14|  [ '第三个预告哈哈哈哈哈A的预告...|
|  14246340| 1871|     24|  [ '第一视角好评，怎么不火Σっ ...|


# sentiment analysis

In [27]:
#convert rdd list to string
from pyspark.sql.functions import udf, col
join_udf = udf(lambda x: ",".join(x))
joinedDF = joinedDF.withColumn("mess_age", join_udf(col("collect_list(mess_age)")))

joinedDF.createOrReplaceTempView("joinedDF")
joinedDF_p = spark.sql("SELECT video_code,view,mess_age from joinedDF limit 50")


# 
joinedDF_p




DataFrame[video_code: bigint, view: bigint, mess_age: string]

In [28]:
joinedDF_p.show()

+----------+-----+--------------------+
|video_code| view|            mess_age|
+----------+-----+--------------------+
|  14245770|  274| '粉嫩嫩的好可爱呀ヾ≧O≦〃嗷~...|
|  14246196|   51| '今天终于补考完了，一会下单，半音阶'|
|  14247593| 1018|   '原来是十诫石板是这么用的啊……'|
|  14245696|   22|          '扎zn了，老fe'|
|  14246555|  126|           '第一嘻嘻･∀･'|
|  14246820| 1439| '哇⊙o⊙哇', '听up主的话...|
|  14246972|  459| '学生党没时间看直播啊。', '...|
|  14247195|  310|               '第一？'|
|  14247454| 1792| '封面是啥', '厉害厉害', ...|
|  14245959|14472| '觉得大胖像黄渤，我老公说像孙红...|
|  14246902|   62|               '第一！'|
|  14247336|  799| '挨炮主，你以后会一直录制走a怪...|
|  14246764|  915| '料太多卷不起来的我心情复杂又扣...|
|  14246814|  275| 'up我们做朋友吧_´□`」 ∠...|
|  14246007|  884| '现版本三大弱c猴子小鱼人小黑，...|
|  14246223| 1106| '第三个预告哈哈哈哈哈A的预告里...|
|  14246340| 1871| '第一视角好评，怎么不火Σっ °...|
|  14246651|  471| '今天姐弟俩被******游客扔...|
|  14246917|  180| '这首歌以前不记得是哪个字幕组翻...|
|  14247035|   14|              '日常沙发'|
+----------+-----+--------------------+
only showing top 20 rows



In [29]:
# pip install jieba; conda install -c conda-forge jieba
import jieba
# view=joinedDF.rdd.map(lambda s : ReduceRate(s[1]))
view=joinedDF_p.rdd.map(lambda s : s[1])
messa=joinedDF_p.rdd.map(lambda s : s[2])
words=messa.map(lambda w:"/".join(jieba.cut_for_search(w))).map(lambda line: line.split("/"))



In [30]:
# messa.collect()
words.collect()
# df = spark.createDataFrame(reply)

# df.createOrReplaceTempView("reply")

[[' ',
  "'",
  '粉',
  '嫩嫩',
  '嫩嫩的',
  '好',
  '可爱',
  '呀',
  'ヾ',
  '≧',
  'O',
  '≦',
  '〃',
  '嗷',
  '~',
  "'",
  ',',
  ' ',
  "'",
  '6666',
  "'",
  ',',
  ' ',
  "'",
  '第五',
  '｡',
  '･',
  'ω',
  '･',
  '｡',
  "'",
  ',',
  ' ',
  "'",
  '偶吼',
  '，',
  '第四',
  '°',
  '∀',
  '°',
  'ﾉ',
  "'",
  ',',
  ' ',
  "'",
  'emmmmm',
  '。',
  '。',
  '。',
  '想',
  '问',
  '一下',
  '，',
  'up',
  '手里',
  '拿',
  '的',
  '是',
  '什么',
  '东西',
  '？',
  '？',
  '=',
  '・',
  'ω',
  '・',
  '=',
  "'",
  ',',
  ' ',
  "'",
  '第二',
  'vv',
  "'",
  ',',
  ' ',
  "'",
  '･',
  '∀',
  '･',
  '第一',
  "'"],
 [' ',
  "'",
  '今天',
  '终于',
  '补考',
  '完',
  '了',
  '，',
  '一会',
  '下单',
  '，',
  '半音',
  '音阶',
  '半音阶',
  "'"],
 [' ', "'", '原来', '是', '十诫', '石板', '是', '这么', '用', '的', '啊', '…', '…', "'"],
 [' ', "'", '扎', 'zn', '了', '，', '老', 'fe', "'"],
 [' ', "'", '第一', '嘻嘻', '･', '∀', '･', "'"],
 [' ',
  "'",
  '哇',
  '⊙',
  'o',
  '⊙',
  '哇',
  "'",
  ',',
  ' ',
  "'",
  '听',
  'up',
  '主',
  '的话',
  '，',
 

In [31]:
from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF()
tf = hashingTF.transform(words)
tf.cache()

PythonRDD[149] at RDD at PythonRDD.scala:48

In [33]:
# 计算 TF-IDF 矩阵
idfModel = IDF().fit(tf)
tfidf = idfModel.transform(tf)
zipped=view.zip(tfidf)

In [ ]:
tfidf

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55586)
Traceback (most recent call last):
  File "D:\Programs\spark-2.2.0-bin-hadoop2.6\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Programs\spark-2.2.0-bin-hadoop2.6\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55586)
Traceback (most recent call last):
  File "D:\Programs\spark-2.2.0-bin-hadoop2.6\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empt

In [39]:
from pyspark.mllib.regression import LabeledPoint
data=zipped.map(lambda line:LabeledPoint(line[0],line[1]))
training, test = data.randomSplit([0.6, 0.4], seed = 0)

# predictionAndLabel = test.map(lambda p : (NBmodel.predict(p.features), p.label))
# accuracy = 1.0 * predictionAndLabel.filter(lambda x: 1.0 \
#     if x[0] == x[1] else 0.0).count() / test.count()


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55586)
Traceback (most recent call last):
  File "D:\Programs\spark-2.2.0-bin-hadoop2.6\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1021, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Programs\spark-2.2.0-bin-hadoop2.6\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "D:\Programs\spark-2.2.0-bin-hadoop2.6\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1025, in send_command
    "Error while sending", e, proto.ERROR_ON_SEND)
py4j.protocol.Py4JNetworkError: Error while sending

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Prog

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:55586)

In [ ]:
from pyspark.mllib.classification import NaiveBayes
# # from pyspark.ml.classification import NaiveBayes
NBmodel = NaiveBayes.train(training, 1.0)
